<h1 style="background-color:#239B56;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
✍ הסבת גבולות תכניות לצרכי רישום בשטח לא מוסדר לבנק"ל מודרני ✍ 

In [ ]:
print(' Unregistered Tazar borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
from configs import CNFG
from helpers import assign_ProcessName, assign_BlockName

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('   Reading inputs')

<h1 style="text-align:right;font-size:100%">
שכבת גבולות תצ"ר בשטח לא מוסדר

In [ ]:
unreg_tazar_cols = ['TALAR_NUM', 'TALAR_YEAR', 'SURVEYOR_ID', 'GUSH_NUM', 'GUSH_SUFFIX', 'SURVEY_DATE', 'geometry']

unreg_tazar_layer = gpd.read_file(CNFG.CADSDE + 'TAZAR_BORDER_LOMUSDAR.gdb', layer = 'TAZAR_BORDER_LOMUSDAR', include_fields = unreg_tazar_cols)

<h1 style="text-align:right;font-size:100%">
טבלת נתוני תצ"ר

In [ ]:
talar = pd.read_csv(CNFG.Oracle + 'TALAR.csv', usecols = ['TALAR_ID', 'TALAR_NUM', 'TALAR_YEAR', 'STATUS'], encoding = 'utf-8')

<h1 style="text-align:right;font-size:100%">
שכבה מקומית של גבולות תצ"ר ללא מקור, בשטח לא מוסדר

In [ ]:
UnkownUnregiteredazarBorders = gpd.read_file(CNFG.Local + 'UnkownUnregiteredazarBorders.gpkg', layer = 'UnkownUnregiteredazarBorders')

<h1 style="text-align:right;font-size:100%">
שכבת גבולות גושי קדסטר מודרניים

In [ ]:
Blocks = pd.read_csv(CNFG.CSVPath + 'BlockUniqueIDs.csv', usecols = ['Name', 'GlobalID', 'IsTax'])

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('   Cleaning data')

In [ ]:
print('   Data conversion')

In [ ]:
unreg_tazar_layer = pd.concat([unreg_tazar_layer, UnkownUnregiteredazarBorders], ignore_index = True)

<h1 style="text-align:right; font-size:100%">
הוספת שדה מקשר של מספר אליפסה מלא לצורך מיזוג

In [ ]:
unreg_tazar_layer = unreg_tazar_layer.dropna(subset=['TALAR_NUM', 'TALAR_YEAR'])


unreg_tazar_layer[['TALAR_NUM', 'TALAR_YEAR']] = unreg_tazar_layer[['TALAR_NUM', 'TALAR_YEAR']].astype(int)


bad_years_cond = unreg_tazar_layer['TALAR_YEAR'] < 1000
unreg_tazar_layer.loc[bad_years_cond, "TALAR_YEAR"] = unreg_tazar_layer['TALAR_YEAR'] + 2000
del bad_years_cond


unreg_tazar_layer = assign_ProcessName(unreg_tazar_layer, 'TALAR_NUM', 'TALAR_YEAR', drop=False)

<h1 style="text-align:right; font-size:100%">
הוספת שדה מקשר של מספר גוש מלא לצורך מיזוג

In [ ]:
unreg_tazar_layer = assign_BlockName(unreg_tazar_layer, 'GUSH_NUM', 'GUSH_SUFFIX')

<h1 style="text-align:right; font-size:100%">
השלמת ערכים חסרים בתאריך המדידה והפקת שנת המדידה

In [ ]:
unreg_tazar_layer['SURVEY_DATE'] = pd.to_datetime(unreg_tazar_layer['SURVEY_DATE'])
unreg_tazar_layer['SurveyYear'] = unreg_tazar_layer['SURVEY_DATE'].dt.year
unreg_tazar_layer['SurveyYear'] = unreg_tazar_layer['SurveyYear'].fillna(unreg_tazar_layer['TALAR_YEAR'])
unreg_tazar_layer['SurveyYear'] = unreg_tazar_layer['SurveyYear'].astype('Int32')


unreg_tazar_layer.drop(columns = ['SURVEY_DATE', 'TALAR_YEAR', 'TALAR_NUM'], inplace=True)

<h1 style="text-align:right; font-size:100%">
הסרת רשומות כפולות לפי מספר אליפסה וגיאומטריה זהים

In [ ]:
unreg_tazar_layer = unreg_tazar_layer[~unreg_tazar_layer.duplicated(subset=['ProcessName', 'geometry'], keep = 'first')].sort_values('ProcessName')
unreg_tazar_layer = unreg_tazar_layer.drop_duplicates(subset=['ProcessName', 'SURVEYOR_ID'], keep = 'first')

In [ ]:
talar = talar.dropna()
talar = talar.astype(int)
talar = assign_ProcessName(talar, 'TALAR_NUM', 'TALAR_YEAR')
talar.rename(columns = {'TALAR_ID' : 'ORACLE_ID'}, inplace = True)

talar = talar[talar['ProcessName'].isin(unreg_tazar_layer['ProcessName'])]

unreg_tazar_layer = unreg_tazar_layer.merge(talar, on = 'ProcessName', how = 'left')

<h1 style="text-align:right; font-size:100%">
איחוד עם טבלת מזהי גושים לקבלת מספר מזהה של הגוש

In [ ]:
CadBlocks = Blocks[Blocks['IsTax'] == 0].drop(columns = 'IsTax')\
                                        .rename(columns = {'GlobalID':'BlockUniqueID', 'Name':'BlockName'})

TaxBlocks = Blocks[Blocks['IsTax'] == 1].drop(columns = 'IsTax')\
                                        .rename(columns = {'GlobalID':'TaxBlockUniqueID', 'Name':'BlockName'})\
                                        .drop_duplicates('BlockName', keep = False) #גושי שומא מפוצלים לא יישויכו לתצ"ר


unreg_tazar_layer = unreg_tazar_layer.merge(CadBlocks, on = 'BlockName', how = 'left')\
                                     .merge(TaxBlocks, on = 'BlockName', how = 'left')\
                                     .drop(columns = 'BlockName')

unreg_tazar_layer['BlockUniqueID'] = unreg_tazar_layer['BlockUniqueID'].fillna(unreg_tazar_layer['TaxBlockUniqueID'])
unreg_tazar_layer.drop(columns = 'TaxBlockUniqueID', inplace = True)

del [Blocks, CadBlocks, TaxBlocks]

<h1 style="text-align:right; font-size:100%">
יצירה והגדרת שדה של מקור הנתונים

In [ ]:
CAD_cond = unreg_tazar_layer['SurveyYear'] <= 2015
unreg_tazar_layer['DataSource'] = np.where(CAD_cond, 5, 4)  # CAD source:5, SRV source:4
del CAD_cond

<h1 style="text-align:right; font-size:100%">
מיפוי ערכי סטאטוסים

In [ ]:
statusDict = {0 : 1, #לא ידוע - קליטת קבצים
              1 : 1, #קליטת קבצים - קליטת קבצים
              2 : 2, #בביקורת - בביקורת
              3 : 4, #כשרה לרישום - כשרה לרישום
              4 : 5, #רשומה - רשומה
              5 : 9, #תצ"ר מבוטל - מבטל
              6 : 3, # בתיקונים - הוחזר למודד
              None : 1} #ללא ערך - קליטת קבצים
unreg_tazar_layer['STATUS'] = unreg_tazar_layer['STATUS'].map(statusDict)
unreg_tazar_layer['STATUS'] = unreg_tazar_layer['STATUS'].fillna(1)

del statusDict

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('   Defining new Layer')

In [ ]:
new_columns = np.dtype([
                        ("geometry", gpd.array.GeometryDtype),       
                        #("GlobalID", str),
                        ("ProcessName", str),
                        ("ProcessType", int),
                        ("Status", int),
                        ("BlockUniqueID", int),
                        ("GeodeticNetwork", int),
                        ("SurveyorLicenseID", int),
                        ("DataSource", int),
                        ("ORACLE_ID", int),
                       #("PlanName", str),   
                       #("Shape_Length", float),
                       #("Shape_Area", float),
                        ])

unreg_tazar_layer_new = gpd.GeoDataFrame(np.empty(0, dtype=new_columns), geometry = 'geometry', crs = 2039)

del new_columns

<h1 style="text-align:right; font-size:100%">
איכלוס שדות

In [ ]:
unreg_tazar_layer_new['geometry'] = unreg_tazar_layer['geometry']


unreg_tazar_layer_new['ProcessName'] = unreg_tazar_layer['ProcessName'].astype('str')


unreg_tazar_layer_new['ProcessType'] = 11  #תצ"ר בשטח לא מוסדר


unreg_tazar_layer_new['DataSource'] = unreg_tazar_layer['DataSource'].astype('Int64')


unreg_tazar_layer_new['Status'] = unreg_tazar_layer['STATUS'].astype('Int32')


geodeticNetDict = {range(0, 1998)    : 1,      # רשת ישראל הישנה
                   range(1999, 2007) : 2,      # רשת ישראל החדשה
                   range(2008, 99999): 3}      # רשת ישראל התקפה
unreg_tazar_layer_new['GeodeticNetwork'] = unreg_tazar_layer['SurveyYear'].apply(lambda x: next((v for k, v in geodeticNetDict.items() if x in k), 0))
unreg_tazar_layer_new['GeodeticNetwork'] = unreg_tazar_layer_new['GeodeticNetwork'].astype('Int64')


unreg_tazar_layer_new['SurveyorLicenseID'] = unreg_tazar_layer['SURVEYOR_ID'].astype('Int64')

#unreg_tazar_layer_new['PlanName'] = None

unreg_tazar_layer_new['BlockUniqueID'] = unreg_tazar_layer['BlockUniqueID']

unreg_tazar_layer_new['ORACLE_ID'] = unreg_tazar_layer['ORACLE_ID'].astype('Int64')


del unreg_tazar_layer

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;">

<h1 style="text-align:right; font-size:100%;">     
פיצול התוצאות לפי סוגי גיאומטריה שונים (פעולה טכנית כי לא ניתן לכתוב קובץ סוגי גיאומטריה שונים באותה שכבה)    
<h1 style="text-align:right; font-size:100%;"> 
שמירת שכבה גיאוגרפית 

In [ ]:
print('   Exporting results')

In [ ]:
Polygons      = unreg_tazar_layer_new[unreg_tazar_layer_new['geometry'].geom_type == 'Polygon']

MultiPolygons = unreg_tazar_layer_new[unreg_tazar_layer_new['geometry'].geom_type == 'MultiPolygon']

In [ ]:
Polygons.to_file(CNFG.Results, layer = 'PT11', driver = 'OpenFileGDB', engine = 'pyogrio', mode = 'w')
MultiPolygons.to_file(CNFG.Results, layer = 'PT11', driver = 'OpenFileGDB', engine = 'pyogrio', mode = 'a')